# Load SQLite DB

In [1]:
import pandas as pd
from paths import PF_DB_PATH
db_path = f'sqlite:///{PF_DB_PATH}'

In [7]:
%%capture
%load_ext sql
%env DATABASE_URL= $db_path

# Querys

#### How is property type distributed in the dataset?

In [10]:
%%sql 
SELECT
    property_type AS propertyType,
    COUNT(property_type) AS total,
    PRINTF("%.1f%",
        100*CAST(COUNT(property_type) AS real) / (
            SELECT
                COUNT(*)
            FROM
                record
        )
    ) AS proportion
FROM
    record
GROUP BY
    property_type
ORDER BY
    total DESC

 * sqlite:///D:\_CORE\static\dataset\Australia\pricefinder\pricefinder_db.sqlite
Done.


propertyType,total,proportion
House,349058,47.7%
Unit,340298,46.5%
Other,22206,3.0%
Vacant Land,12576,1.7%
Commercial,6310,0.9%
Industrial,1104,0.2%


#### How much have average house prices changes in the last 5 years?

In [4]:
%%sql
WITH houseAvg AS (
    SELECT
        CAST(strftime('%Y', record_date) AS decimal) AS year,
        AVG(price) AS price
    FROM
        record
    WHERE
        property_type = 'House'
    GROUP BY
        year
)
SELECT
    year,
    PRINTF("$%,d", price) AS avgPrice,
    PRINTF("%.1f%",
        100*(price / lag(price, 1) over(ORDER BY year)-1)
    ) AS changeFromPreviousYear
FROM
    houseAvg
WHERE
    year < 2022
ORDER BY
    year DESC
LIMIT
    5

 * sqlite:///D:\_CORE\static\dataset\Australia\pricefinder\pricefinder_db.sqlite
Done.


year,avgPrice,changeFromPreviousYear
2021,"$1,649,125",35.2%
2020,"$1,220,014",23.5%
2019,"$987,961",-1.2%
2018,"$999,697",-5.9%
2017,"$1,062,668",10.8%


In [5]:
%%sql


UsageError: %%sql is a cell magic, but the cell body is empty. Did you mean the line magic %sql (single %)?
